In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from sklearn.decomposition import TruncatedSVD

In [2]:
df=pd.read_csv('PBL5recommendationdata.csv',encoding='latin-1')

In [3]:
df.head()

,Customers.id,Customers.fname,Customers.lname,Customers.company,Customers.create_date,Customers.status,Customers.mailing,Customers.reminders,Customers.tax_exempt,Customers.account_id,Customers.sales_rep,Customers.rewards,Customers.profile_id,Customers.last_modified,Customers.customer_type,Orders.id,Orders.customer_id,Orders.fname,Orders.lname,Orders.company,Orders.order_number,Orders.reorder_id,Orders.external_source,Orders.external_id,Orders.currency,Orders.sales_rep,Orders.subtotal,Orders.tax,Orders.shipping,Orders.coupon_id,Orders.coupon_amount,Orders.gift_id,Orders.gift_amount,Orders.fee_name,Orders.fee_amount,Orders.discount_name,Orders.discount_amount,Orders.total,Orders.balance_due,Orders.shipping_carrier,Orders.shipping_method,Orders.shipping_trans,Orders.shipping_flags,Orders.weight,Orders.tracking,Orders.payment_status,Orders.payment_date,Orders.payment_user,Orders.payment_type,Orders.payment_method,Orders.payment_amount,Orders.purchase_order,Orders.payment_id,Orders.payment_code,Orders.payment_ref,Orders.status,Orders.placed_date,Orders.updated_date,Orders.shipped_date,Orders.comments,Orders.notes,Orders.registry_id,Orders.gift_message,Orders.website,Orders.mailing,Orders.flags,Orders.partial_ship,Orders.customer_type,Order_Items.id,Order_Items.parent,Order_Items.product_id,Order_Items.product_name,Order_Items.attributes,Order_Items.attribute_names,Order_Items.attribute_prices,Order_Items.qty,Order_Items.price,Order_Items.cost,Order_Items.registry_item,Order_Items.related_id,Order_Items.reorder_frequency,Order_Items.account_id,Order_Items.flags,Products.id,Products.status,Products.product_type,Products.template,Products.vendor,Products.import_id,Products.name,Products.display_name,Products.menu_name,Products.list_price,Products.price,Products.sale_price,Products.cost,Products.flags,Products.left_flag,Products.right_flag,Products.last_modified,Products.taxable,Products.shopping_gtin,Products.shopping_brand,Products.shopping_mpn,Products.shopping_gender,Products.shopping_color,Products.shopping_age,Products.shopping_flags,Products.amazon_asin,Products.amazon_type,Products.amazon_item_type,Products.amazon_price,Products.google_shopping_id,Products.google_shopping_type,Products.google_shopping_cat,Products.google_adwords,Products.shopping_cat,Products.shopping_type,Products.pricegrabber_cat,Products.shopzilla_cat,Products.thefind_cat,Products.quickbooks_id,Products.qb_edit_sequence,Products.price_break_type,Products.price_breaks,Products.short_description,Products.long_description,Products.websites,Products.video,Products.audio,Products.seo_title,Products.seo_description,Products.seo_keywords,Products.seo_header,Products.seo_footer,Products.seo_url,Products.seo_category,Products.unit,Products.packaging,Products.display_packaging,Products.multiple,Products.length,Products.width,Products.height,Products.rx,Products.latex,Products.upc,Products.msds_link,Products.msds_label,Products.lit_link,Products.lit_label,Products.hcpcs,Products.case_qty,Products.markup,Products.override_markup,Products.notes,Products.import_flags,Products.map_price,Products.features_title,Products.warranty,Products.hygienic,Products.default_quantity,Products.user_size,Products.assembly,Products.installation,Products.shipping_length,Products.shipping_width,Products.shipping_height,Products.shipping_weight,Products.handling_time,Products.rotation_link,Products.google_shopping_label,Products.product_option,Products.size,Products.material,Products.arm_style,Products.leg_style,Products.seat_size,Products.family_id,Products.saved_status,Products.freight_cost
0,797,Christy,Dill,Company0,1426018724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1437764306,0.0,3758,797,Christy,Dill,Company0,3758,NaN,NaN,NaN,USD,NaN,57.20,0.0,9.95,13.0,2.86,NaN,NaN,NaN,NaN,NaN,NaN,64.29,NaN,fedex,11|Ground,NaN,NaN,NaN,5.7204E+14,3.0,1.426019e+09,NaN,authorize.net,NaN,64.29,NaN,6993607863,510142,NaN,1,1426019099,1.438868e+09,1.426101e+09,NaN,Insured By Eye4Fraud,NaN,NaN,NaN,NaN,NaN,NaN,0.0,528

In [4]:
df['Orders.shipping_method'].unique()

array(['11|Ground', '0|Standard Shipping', '-1|Free Shipping', nan,
       'NATIONAL DELIVERY SYSTEMS', 'LAND AIR OF NEW ENGLAND',
       '0|Free Shipping', 'INTERNATIONAL FEDEX',
       'SOUTHWESTERN MOTOR TRNAS', 'WILSON TRUCKING CORP',
       "HOWARD'S EXPRESS, INC.", 'MEDTRANS', 'NATIONAL DELIVERY',
       'FEDERAL EXPRESS CORP.', 'ROADWAY EXPRESS'], dtype=object)

In [5]:
df['Products.name'].nunique()

1391

In [6]:
df['Customers.company'].value_counts()
#Company145 is the company that makes the most purchases, but we cannot say that this company makes the most money in the store because
# we don't know how many times he bought which product at a time and how much money he paid we will look at it below

Company145    27
Company130    17
Company59     15
Company52     13
Company171    12
              ..
Company163     1
Company162     1
Company160     1
Company157     1
Company425     1
Name: Customers.company, Length: 425, dtype: int64

In [7]:
df.groupby('Customers.company')['Orders.total'].value_counts().sort_values(ascending=False)

Customers.company  Orders.total
Company264         160.29          9
Company159         938.18          8
Company145         270.64          8
Company59          1038.30         6
Company89          952.18          6
                                  ..
Company230         233.82          1
Company229         258.70          1
Company228         65.96           1
Company226         38.51           1
Company99          51.39           1
Name: Orders.total, Length: 536, dtype: int64

In [8]:
df.groupby('Customers.company')['Order_Items.qty'].value_counts().sort_values(ascending=False)

Customers.company  Order_Items.qty
Company145         1                  27
Company59          30                 11
Company86          10                  9
Company6           2                   7
Company159         1                   7
                                      ..
Company24          1                   1
Company238         5                   1
                   1                   1
Company237         1                   1
Company99          2                   1
Name: Order_Items.qty, Length: 507, dtype: int64

In [9]:
df['Products.name'].value_counts().sort_values(ascending=False)
# The most purchased product is Emesis Bags,Blue,36,000 OZ

Emesis Bags,Blue,36.000 OZ                                                                103
MoliCare Disposable Super Plus Briefs,27"-47"                                              71
Aluminum Transport Chair with 12" Wheels,Red,F: 8   R: 12                                  67
Aluminum Transport Chair with 12" Wheels,Blue,F: 8   R: 12                                 67
K1 Basic Extra-Wide Wheelchairs                                                            54
                                                                                         ... 
Combination Plastic Transfer Bench with Commode Opening                                     1
Accu-Therm Insulated Hot Pack                                                               1
CanDo&reg; Inflatable Exercise Ball - Orange - 22" (55 cm)                                  1
CanDo&reg; Antimicrobial Theraputty&reg; Exercise Material - 2 oz -6-piece (tan-black)      1
Spitfire EX Compact Travel Power Mobility Scooter, 4 Wheel  

In [10]:
df.groupby('Products.name')['Order_Items.qty'].value_counts().sort_values(ascending=False)

Products.name                                               Order_Items.qty
Emesis Bags,Blue,36.000 OZ                                  1                  81
Aluminum Transport Chair with 12" Wheels,Red,F: 8   R: 12   1                  66
Aluminum Transport Chair with 12" Wheels,Blue,F: 8   R: 12  1                  66
Silent Knight Pill Crushers                                 1                  50
K1 Basic Extra-Wide Wheelchairs                             1                  50
                                                                               ..
FitRight Restore Briefs,48"-58"                             2                   1
FitRight Restore Briefs,32"-42"                             1                   1
FitRight Plus Briefs,60"-69"                                4                   1
                                                            2                   1
sEMG - single use uni-gel electrodes only, case of 100      1                   1
Name: Order_Items.qty,

In [11]:
df['Products.shopping_brand'].value_counts().sort_values(ascending=False)
# The most shopped brand is MEDLINE

MEDLINE                             2672
Drive Medical                        535
HARTMANN-CONCO INC                   122
CanDo&reg;                           113
Fabrication Enterprises Inc.          85
LINKS MEDICAL PRODUCTS                53
Wenzelite                             36
Lifestyle Essentials                  24
Baseline&reg;                         24
Relief Pak&reg;                       19
the Cuff&reg;                         17
Dynarex                               16
ITA-MED                               16
Dycem                                 15
FabLife&trade;                        14
GERITREX CORPORATION                  14
WESTMED INC                           14
ColPaC&reg;                           13
Mason Medical                         13
CYPRESS PHARMACEUTICAL INC            10
EvenUp                                 9
MAXAR                                  8
NeckPro&trade;                         7
Medquip                                7
GABRIALLA       

In [12]:
df.groupby('Products.name')['Order_Items.reorder_frequency'].value_counts().sort_values(ascending=False)

Products.name                                                         Order_Items.reorder_frequency
Protect Extra Protective Underwear,Medium                             1.0                              2
FitRight Plus Briefs,60"-69"                                          1.0                              2
Capri Bladder Control Pads,2.75" X 9.75"                              1.0                              1
Protection Plus Disposable Underpads,Blue,24" X 17"                   2.0                              1
Remedy Phytoplex Hydrating Cleansing Gel,16.00 OZ                     1.0                              1
Remedy Olivamine Foaming Body Cleanser - 9 ounce - Pack of 3 bottles  3.0                              1
ReadyBath Total Body Cleansing Standard Weight Washcloths             4.0                              1
                                                                      2.0                              1
Protection Plus Polymer-Filled Underpads,Green,30" X 30"    

In [13]:
df['Orders.payment_method'].unique()

array([nan, 'Credit Card', 'PayPal', 'Check', 'Cash'], dtype=object)

In [14]:
df['Orders.payment_method'].value_counts().sort_values(ascending=False)


Credit Card    373
PayPal         318
Check           29
Cash             2
Name: Orders.payment_method, dtype: int64

In [15]:
df['Order_Items.product_name'].unique()

array(['Basic Rollators, Green', 'Urinary Drain Bags,0.000',
       'SensiCare Nitrile Exam Gloves, Blue, XX-Large', ...,
       'Relief Pak&reg; Moist Heat Pack - Spine - 10" x 24"',
       'Spitfire EX Travel 4-Wheel Mobility Scooter 21AH Bettery',
       'Medline Sterile Alcohol Prep Pads, Large'], dtype=object)

In [16]:
df['Products.quickbooks_id']=[str(i)[:8] for i in df['Products.quickbooks_id'].values.tolist()]

In [17]:
df['Products.quickbooks_id'].value_counts().sort_values(ascending=False)

nan         466
8000115D     88
80001C24     71
8000023B     67
8000023C     67
           ... 
80007AC8      1
80007555      1
80008A0A      1
8000631C      1
800082C6      1
Name: Products.quickbooks_id, Length: 1522, dtype: int64

In [18]:
df['Orders.shipping_method'].unique()

array(['11|Ground', '0|Standard Shipping', '-1|Free Shipping', nan,
       'NATIONAL DELIVERY SYSTEMS', 'LAND AIR OF NEW ENGLAND',
       '0|Free Shipping', 'INTERNATIONAL FEDEX',
       'SOUTHWESTERN MOTOR TRNAS', 'WILSON TRUCKING CORP',
       "HOWARD'S EXPRESS, INC.", 'MEDTRANS', 'NATIONAL DELIVERY',
       'FEDERAL EXPRESS CORP.', 'ROADWAY EXPRESS'], dtype=object)

In [19]:
df['Products.amazon_item_type'].unique()

array(['health-care-products', nan], dtype=object)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4194 entries, 0 to 4193
Columns: 181 entries, Customers.id to Products.freight_cost
dtypes: float64(98), int64(10), object(73)
memory usage: 5.8+ MB


In [21]:
miss=df.isnull().sum()
miss[miss>0].sort_values()

Order_Items.cost                     3
Orders.payment_status                5
Orders.payment_amount               18
Orders.payment_date                 18
Orders.updated_date                 23
Order_Items.product_id              43
Products.last_modified             157
Products.cost                      157
Products.name                      157
Products.import_id                 157
Products.vendor                    157
Products.status                    157
Products.id                        157
Products.seo_url                   157
Products.price                     158
Products.shopping_mpn              160
Products.shopping_brand            162
Products.list_price                177
Products.amazon_item_type          179
Orders.customer_type               183
Products.google_shopping_cat       183
Products.seo_title                 183
Products.long_description          186
Orders.shipping                    195
Products.shipping_width            196
Products.shipping_length 

In [22]:
df['Products.price'].max(),df['Products.price'].min()

(1899.0, 1.54)

In [23]:
df['Orders.total'].describe()

count    4194.000000
mean      137.192818
std       222.898475
min         2.890000
25%        44.110000
50%        80.510000
75%       143.855000
max      4627.950000
Name: Orders.total, dtype: float64

In [24]:
abs(df.corr()['Orders.total'].sort_values(ascending=False))

Orders.weight                     1.000000
Orders.total                      1.000000
Orders.subtotal                   0.997013
Customers.tax_exempt              0.994398
Orders.payment_amount             0.992953
Orders.discount_amount            0.928716
Products.map_price                0.925721
Products.amazon_price             0.716534
Products.sale_price               0.637339
Orders.fee_amount                 0.628024
Orders.coupon_amount              0.561841
Orders.external_id                0.489013
Orders.balance_due                0.482971
Order_Items.price                 0.428587
Order_Items.cost                  0.425841
Products.cost                     0.422208
Products.price                    0.412019
Products.shipping_weight          0.390454
Order_Items.qty                   0.384124
Products.list_price               0.380704
Orders.partial_ship               0.236900
Orders.tax                        0.199910
Products.shipping_width           0.177553
Products.go

In [25]:
abs(df.corr()['Products.price'].sort_values(ascending=False))

Products.price                    1.000000
Products.map_price                0.999913
Customers.tax_exempt              0.997547
Products.cost                     0.986842
Order_Items.cost                  0.983117
Order_Items.price                 0.982685
Products.sale_price               0.966714
Products.amazon_price             0.948921
Products.list_price               0.905323
Products.shipping_weight          0.845600
Products.shipping_length          0.506309
Products.google_shopping_label    0.434385
Products.shipping_height          0.412666
Orders.total                      0.412019
Products.height                   0.406576
Orders.payment_amount             0.401927
Orders.subtotal                   0.400565
Products.shipping_width           0.391986
Orders.balance_due                0.352717
Products.length                   0.333933
Orders.coupon_amount              0.274981
Orders.tax                        0.258188
Products.shopping_gtin            0.246500
Products.ov

### MATRIX FACTORIZATION

In [26]:
matrix_=df.pivot_table(index='Products.id',columns='Products.name',values='Orders.total').fillna(0)
matrix_.shape

(1710, 1391)

In [27]:
matrix_.head(10)

Products.name,1 Box Of Medline Commode Liners with Absorbent Pad,"1-Ply Blockade AngelStat Surgical Gowns,Ceil Blue,Extra Large","1/2"" self-adhesive hook material, 25 yard dispenser box, white","1/2"" self-adhesive loop material, 25 yard dispenser box, white","100% Cotton Unisex Reversible Hyperbaric Scrub Tops,Misty Green,MZS","100% Cotton Unisex Reversible Tops,Ceil Blue,MHS","18"" Hamper Stands,Standard","2"" self-adhesive loop material, 10 yard dispenser box, tan","2"" self-adhesive loop material, 25 yard dispenser box, tan","2-Ply Tissue/Poly Professional Towels,Not Applicable",2000 Wheelchairs,"24 Hour Urine Collection Bottle,3000.0 ML","3 Pack, Hospital Receiving Blankets, Baby Blankets, 100% Cotton, 30x40, Elephant","3 Pack, Hospital Receiving Blankets, Baby Blankets, 100% Cotton, 34x40, Footprints",3 Wheel Walker Rollator with Basket Tray and Pouch,"3"" Wheels For Walkers","3/4"" Guardian Utility Tips,Gray","5"" Wheels For Walkers",6 Cylinder Oxygen Carrier,6-spring hand grip (each),"8"" Rear Walker Wheel Attachment","A&D Ointment by H & H Laboratories Inc,384.00 ML",Accu-Therm Hot/Cold Gel Packs,Accu-Therm Insulated Hot Pack,Accu-Therm Non-Insulated Hot Pack,"Accutouch Chemo Nitrile Exam Gloves,Blue,Medium","Active Liquid Protein Nutritional Supplement,30.000 OZ",AcuForce&reg; 7.0 Massage Tool,"Addipak Sterile Saline Solution,3.00 ML",Adjustable Angle Rotating Suction Cup Grab Bar,Adjustable Height Bathtub Grab Bar Safety Rail,Adjustable Height Home Bed Assist Handle,"Adjustable Height Rollator with 6"" Wheels",Adjustable Height Teak Bath Bench Stool,Adjustable Lightweight Folding Cane with Gel Hand Grip,Adult Bariatric Crutches,"AdvanTrode&reg; Elite Electrode, 2"" round, tan tricot, 40/box","Advantage Magnetic Patient Alarms,Yellow",Advantage Therapeutic Homecare Foam Mattress,Aero-Pulse Pressure Pads,Aeroflow II Wheelchair Cushions,Aeromist Plus Nebulizer Compressor with Disposable Nebulizer Kit,All Purpose Rolling Shopping Utility Cart,"Aloetouch 3G Powder-Free Latex-Free Synthetic Exam Gloves,Large","Aloetouch 3G Powder-Free Latex-Free Synthetic Exam Gloves,Medium","Aloetouch 3G Powder-Free Latex-Free Synthetic Exam Gloves,Small","Aloetouch 3G Powder-Free Latex-Free Synthetic Exam Gloves,X-Large","Aloetouch Extended Cuff Chemo Nitrile Exam Gloves,Green,Medium","Aloetouch Extended Cuff Chemo Nitrile Exam Gloves,Green,Small","Aloetouch Extended Cuff Chemo Nitrile Exam Gloves,Green,X-Large",Aloetouch PROTECT Dimethicone Skin Protectant Wipes,Aloetouch Personal Cleansing Wipes,"Aloetouch Powder-Free Latex Exam Gloves,Green,X-Small",Aloetouch SELECT Premium Spunlace Personal Cleansing Wipes,Aloetouch Sensitive Personal Cleansing Baby Wipes,"Aloetouch Ultra IC Powder-Free Latex-Free Synthetic Exam Gloves,Large","Aloetouch Ultra IC Powder-Free Latex-Free Synthetic Exam Gloves,Medium",Aluminum Bath Benches with Back,Aluminum Bath Benches without Back,Aluminum Folding Commode with Padded Armrests,Aluminum Forearm Crutches,"Aluminum Quad Canes,Black","Aluminum Quad Canes,Chrome",Aluminum Rollator with Fold Up and Removable Back Support and Padded Seat,Aluminum Shower Commode Mobile Chair,"Aluminum Transport Chair with 12"" Wheels,Blue,F: 8 R: 12","Aluminum Transport Chair with 12"" Wheels,Red,F: 8 R: 12","Aluminum Transport Chair with 8"" Wheels,Blue,F: 8 R: 8","Aluminum Transport Chair with 8"" Wheels,Red,F: 8 R: 8","AngelStat Ladies Elastic with Draw Cord Scrub Pants,Ceil Blue,NTH","AngelStat Ladies Elastic with Draw Cord Scrub Pants,Hunter Green,NHG","AngelStat Ladies Elastic with Draw Cord Scrub Pants,Navy Blue,NNT","AngelStat Unisex Elastic Waist Cargo Scrub Pants,Hunter Green,NHG","AngelStat Unisex Reversible Drawstring Scrub Pants,Jade Green,NTJ","AngelStat Unisex Reversible Drawstring Scrub Pants,Misty Green,NTZ","AngelStat Unisex Reversible V-Neck Scrub Tops,Misty Green,NTZ",Ankle Prompts for Trekker Gait Trainer,Anti Tippers with Wheels,Antiseptic Towelettes,"Aquasonic&reg; 100 ultrasonic gel, 250ml (8.5 oz) di

In [28]:
X=matrix_.values.T

In [29]:
SVD=TruncatedSVD(n_components=12, random_state=0)
matrix=SVD.fit_transform(X)
matrix.shape

(1391, 12)

In [30]:
corr = np.corrcoef(matrix)
corr.shape

(1391, 1391)

In [31]:
title=matrix_.columns
title_list=list(title)
samia = title_list.index('Emesis Bags,Blue,36.000 OZ')
corr_samia  = corr[samia]
list(title[(corr_samia >= 0.9)])

['Adult Bariatric Crutches',
 'Aloetouch Ultra IC  Powder-Free Latex-Free Synthetic Exam Gloves,Large',
 'Aquasonic&reg; 100 ultrasonic gel, 250ml (8.5 oz) dispenser - each',
 'Bariatric Heavy Duty Bath Bench',
 'Baseline&reg; MMT - Hydraulic - Includes 3 Push, 2 Pull Attachments - digital LCD Gauge - 250 lb Capacity',
 'Basic Specimen Containers,4',
 'Boo-boo Pac&trade; cold pack - blue, Retail box of 10',
 'CAT - Chiropractic Adjusting Tool - Soft Tip replacement only - Pack of 5 Rubber Tips',
 'CURAD Elastic Foam Adhesive Tape,White',
 'CURAD Open-Patella Knee Supports,Black,2X-Large',
 'CURAD Powder-Free Textured Latex Exam Gloves,Beige,Large',
 'CanDo&reg; 2-Layer Round Foam Roller - 6" x 15" - Yellow - Extra-Soft',
 'CanDo&reg; ArmaSport&reg; balance pad, 16" x 20" x 2.5", blue',
 "CanDo&reg; Low Powder Exercise Tubing - 25' roll - Yellow - x-light",
 'CanDo&reg; Pedal Exerciser- Deluxe with LCD monitor',
 'CanDo&reg; VariGrip&reg; hand exerciser - Green, medium',
 'Capri Plus Bl

### Cosine similarity for product description

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(df)

In [33]:
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names())
df

,account_id,amazon_asin,amazon_item_type,amazon_price,amazon_type,arm_style,assembly,attribute_names,attribute_prices,attributes,audio,balance_due,case_qty,comments,company,cost,coupon_amount,coupon_id,create_date,currency,customer_id,customer_type,customers,default_quantity,discount_amount,discount_name,display_name,display_packaging,external_id,external_source,family_id,features_title,fee_amount,fee_name,flags,fname,freight_cost,gift_amount,gift_id,gift_message,google_adwords,google_shopping_cat,google_shopping_id,google_shopping_label,google_shopping_type,handling_time,hcpcs,height,hygienic,id,import_flags,import_id,installation,last_modified,latex,left_flag,leg_style,length,list_price,lit_label,lit_link,lname,long_description,mailing,map_price,markup,material,menu_name,msds_label,msds_link,multiple,name,notes,order_items,order_number,orders,override_markup,packaging,parent,partial_ship,payment_amount,payment_code,payment_date,payment_id,payment_method,payment_ref,payment_status,payment_type,payment_user,placed_date,price,price_break_type,price_breaks,pricegrabber_cat,product_id,product_name,product_option,product_type,products,profile_id,purchase_order,qb_edit_sequence,qty,quickbooks_id,registry_id,registry_item,related_id,reminders,reorder_frequency,reorder_id,rewards,right_flag,rotation_link,rx,sale_price,sales_rep,saved_status,seat_size,seo_category,seo_description,seo_footer,seo_header,seo_keywords,seo_title,seo_url,shipped_date,shipping,shipping_carrier,shipping_flags,shipping_height,shipping_length,shipping_method,shipping_trans,shipping_weight,shipping_width,shopping_age,shopping_brand,shopping_cat,shopping_color,shopping_flags,shopping_gender,shopping_gtin,shopping_mpn,shopping_type,shopzilla_cat,short_description,size,status,subtotal,tax,tax_exempt,taxable,template,thefind_cat,total,tracking,unit,upc,updated_date,user_size,vendor,video,warranty,website,websites,weight,width
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,

In [34]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(df, df))

[[1.  0.5 0.5 ... 0.  0.  0. ]
 [0.5 1.  0.5 ... 0.  0.  0. ]
 [0.5 0.5 1.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 1.  0.5 0.5]
 [0.  0.  0.  ... 0.5 1.  0.5]
 [0.  0.  0.  ... 0.5 0.5 1. ]]
